## Color spaces

### RBG and CMY

The classical color space RGB is a 3 dimensional cube which coordinates are

* r: quantity of red
* g: quantity of green
* b: quantity of blue

Other colors are made by mixing these 3 coordinate. This mixing is an additive: it is as if you supperpose lights:

In [ ]:
IPython.display.Image("assets_signal/additiveColor.png")

This additive color system is the opposite of the substractive one, which we learn at school by mixing painting.

In [ ]:
IPython.display.Image("assets_signal/blend_CMY.png",width=600)

***To you:*** $(1\heartsuit)$. The basic colors of the substractiive system are CMY. What means these 3 letters?

Come back to RGB.  Usualy, each of the coordinates r,g and b are encoded with a 8 bits unsign integer.  So the total number of different colors is ... $(1\heartsuit)$. The set of colors must be seen as a cube discretized.  



In [ ]:
IPython.display.Image("assets_signal/RGB.png",width=600)

### HSV and HSL

But the RGB system is not pratical to chose a color. There exists other systems, based on the psychological perception. Two very famous systems are:

* HSV = HSB : Hue, Saturation (=chroma), Value (=Brightness)
* HSL : Hue, Saturation (=chroma),  Luminance (=Lightness)

In french:

* TSV: Teinte Saturation Valeur
* TSL: Teinte Saturation Luminance


They are also 3 dimensionnal systems, but it is better to represent them with cones:


In [ ]:
IPython.display.Image("assets_signal/HSL.png",width=600)

In [ ]:
IPython.display.Image("assets_signal/HSV.png",width=600)

Roughtly speaking: to get HSV cone, you taje the HSL cone, and you flatten it by descending its top.

In practive, cones are often deformed into cylinders:

In [ ]:
IPython.display.Image("assets_signal/hsl-hsv-comparison.png",width=400)

These color spaces are used by all softwares to help you to choose a color.

By example, here is a classical color picker based on HSV=HSB:



In [ ]:
IPython.display.Image("assets_signal/color-picker.png",width=400)

* The Hue is encoded by an angle, usualy in degree:  
    *  $0^o=360^o$ is  red
    * $120^o$ is green
    * $240^o$ is blue
    
    
* The saturation is the radius of the disk. It is a measure of the 'purity' of the color:
    * $0$ : all  hue are mixed
    * $100$% : hue is pure
    
* Luminance = Lightness:
    * $0$: very dark: so black
    * $100$%: very lightly: so white
    
* Value = Brightnes
   * $0$: very dark: so black
   * $100$%: very bright colors
    
    
***To you:*** $(1\heartsuit)$ Enter "color picker" in google. At the top of the result, you might find a second kind of color picker (see screen shot below). Explain how its work. In particular, what are the x and y axis of the rectangle.
    

In [ ]:
IPython.display.Image("assets_signal/colorPickerRectangle.png",width=600)

### Implementation of conversions

The following implementation was made by [F. Legrand](https://www.f-legrand.fr/scidoc/simul/image/espaceRGB.html).

You can also find all the formulas in [wikipedia](https://en.wikipedia.org/wiki/HSL_and_HSV#From_HSL)

In [ ]:
""" r,g,b could range in [0,1] or in [0,255]. """
def rgb2hsl(rgb):
    r = rgb[0] * 1.0
    g = rgb[1] * 1.0
    b = rgb[2] * 1.0

    Max = max(r, g, b)
    Min = min(r, g, b)
    C = Max - Min

    L = Max
    if L == 0: return [0, 0, 0]

    S = C / L
    if C == 0: return [0, 0, L]

    if Max == r:
        H = 60.0 * (g - b) / C % 360
    elif Max == g:
        H = 120.0 + 60.0 * (b - r) / C
    else:
        H = 240.0 + 60.0 * (r - g) / C

    return [H, S, L]

In [ ]:
def hsl2rgb(hsl):
    H = hsl[0] * 1.0
    S = hsl[1] * 1.0
    L = hsl[2] * 1.0
    C = L * S
    Min = L - C
    if (H > 300) and (H <= 360):
        r = L
        g = Min
        b = g + C * (360.0 - H) / 60
    elif (H >= 0) and (H <= 60):
        r = L
        b = Min
        g = b + C * (H / 60)
    elif (H > 60) and (H <= 120):
        g = L
        b = Min
        r = b + C * (120.0 - H) / 60
    elif (H > 120) and (H <= 180):
        g = L
        r = Min
        b = r + C * (H - 120.0) / 60
    elif (H > 180) and (H <= 240):
        b = L
        r = Min
        g = r + C * (240.0 - H) / 60
    else:
        b = L
        g = Min
        r = g + C * (H - 240.0) / 60
    return [r, g, b]

In [ ]:
"""test: we check the 3 pure colors"""
rgb2hsl([1,0,0]),rgb2hsl([0,1,0]),rgb2hsl([0,0,1])

In [ ]:
"""test: we check the 3 pure colors"""
rgb2hsl([255,0,0]),rgb2hsl([0,255,0]),rgb2hsl([0,0,255])

In [ ]:
"""convertion go and back"""
hsl2rgb(rgb2hsl([0.5,0.5,0.5]))

Let's play with hues, to change the colors of the nature.

In the code below, remark the use of `np.apply_along_axis` which allows to automaticaly vectorize the code. Actually, I do not now if this is slower than if we manualy vectorize the functions `hsl2rgb` and `rgb2hsl`. If someone have time for this comparison, say me!

In [ ]:
img = img_origin.copy()

In [ ]:
%time
img = np.apply_along_axis(rgb2hsl,2,img)
"hue decay"
img[:,:,0] += 60
img[:,:,0] %= 360
img = np.apply_along_axis(hsl2rgb,2,img)

In [ ]:
img = np.array(img,dtype = np.uint8)
plt.imshow(img);

***To you:*** Make $(4\heartsuit)$ an histogram of the hues of the babouin. Bonus $(4\star)$: stack your histogram with a bar, under the xticks, which shows the hues.

### The disk of the color picker

In [ ]:
def disk(L):
    width = 200
    height = 200
    img = np.zeros((height, width, 4))
    dx = 2.0 / (width - 1)
    dy = 2.0 / (height - 1)
    rad2deg = 180.0 / np.pi

    for i in range(height):
        for j in range(width):
            x = -1.0 + j * dx
            y = -1.0 + i * dy
            r = np.sqrt(x * x + y * y)
            if r < 1.0:
                if x == 0:
                    if y > 0:
                        a = 90.0
                    else:
                        a = -90.0
                else:
                    a = np.arctan(y / x) * rad2deg
                if x < 0:
                    a += 180.0
                a %= 360
                rgb = hsl2rgb([a, r, L])
                img[i, j] = np.array([rgb[0], rgb[1], rgb[2], 1.0])
            else:
                img[i, j] = np.array([1.0, 1.0, 1.0, 0.0])

    fig,ax=plt.subplots(figsize=(int(4*L),int(4*L)))
    ax.imshow(img, origin='lower')
    ax.axis("off")

In [ ]:
disk(1.)

In [ ]:
disk(0.8)

In [ ]:
disk(0.6)

In [ ]:
disk(0.4)

***To you:*** $(1\heartsuit)$ Previous functions `hsl2rgb` and `rgb2hsl` were badly named. Do you see why?


***To you:*** Improve the code of `disk`

* $(3\heartsuit)$   Firstly, the autor of this code did not know the function `np.arctan2(y,x)` which compute the angle of the point `(x,y)`

* $(5\heartsuit)$  Secondly, this code  can be vectorized to improve readibility and  performances.

### Visual explanation of the conversions RGB $\to$ HSL


Observe the foolowing screen shots: the cube represent the RBG system. We make some slice of this cube, each slice, can be deform into a disk, and these disks are (roughtly) the HSL disks: the level of the slice correspond to the Lightness.








In [ ]:
IPython.display.Image("assets_signal/cut1.png",width=400)

In [ ]:
IPython.display.Image("assets_signal/cut2.png",width=400)

In [ ]:
IPython.display.Image("assets_signal/cut3.png",width=400)

In [ ]:
IPython.display.Image("assets_signal/cut4.png",width=400)


You can make the slices yourself, and also turn the cube [here](http://www.f-legrand.fr/scidoc/simul/image/espaceRGB.html).




***To you:*** $(1\heartsuit)$ Explain why, when the saturation  S equals 0, the colors are grey.

### To play with


$(5\star)$ Modify the baboin, playing with saturation and Brightness (or value).